In [1]:
%load_ext autoreload
%autoreload 2

from kaggle_speech_recog import *
import math

# Get feature and label matrices

In [2]:
X_vector_len = 16000
train = SpeechList.get_train('train/audio')
XY_train_valid = train.get_spectrogram_X_and_Y(X_vector_len=X_vector_len, take_log=True, split=0.85)

SKIPPED train/audio/_background_noise_/README.md File format b'# Ba'... not understood.


/usr/local/anaconda3/lib/python3.5/site-packages/scipy/io/wavfile.py:273: WavFileWarning: Chunk (non-data) not understood, skipping it.
  WavFileWarning)


# Train
* Hmm, sparse matrix doesnt work

In [3]:
# Need this while testing graph so it gets autoloaded
from kaggle_speech_recog.graphs.useful_tf_graph import *
from kaggle_speech_recog.graphs.spectrogram_2_conv import *

In [4]:
graph_name = 'Spectrogram2Conv'  # Specify graph class name

g_cnfg = Config('graph_01')  # Specify graph configuration name
g_cnfg.X_img_h = XY_train_valid[0].shape[1]
g_cnfg.X_img_w = XY_train_valid[0].shape[2]
g_cnfg.Y_vector_len = XY_train_valid[1].shape[1]

g_cnfg.conv1_n_filters = 16
g_cnfg.conv1_kernel_size = 5
g_cnfg.conv1_mp_size = 5
g_cnfg.conv1_mp_strides = 2

g_cnfg.conv2_n_filters = 32
g_cnfg.conv2_kernel_size = 3
g_cnfg.conv2_mp_size = 3
g_cnfg.conv2_mp_strides = 2

g_cnfg.n_hidden1 = 1800
g_cnfg.n_hidden2 = 50

g_cnfg.lr_initial = 0.1
g_cnfg.lr_decay_steps = 1000000
g_cnfg.lr_decay_rate = 0.7  # 1.0 means no decay

g_cnfg.X_conv_h = math.ceil(math.ceil(g_cnfg.X_img_h / g_cnfg.conv1_mp_strides) / g_cnfg.conv2_mp_strides)
g_cnfg.X_conv_w = math.ceil(math.ceil(g_cnfg.X_img_w / g_cnfg.conv1_mp_strides) / g_cnfg.conv2_mp_strides)
g_cnfg.n_flat = g_cnfg.X_conv_h * g_cnfg.X_conv_w * g_cnfg.conv2_n_filters

t_cnfg = Config('run_01')  # Specify train configuration name
t_cnfg.ckp_dir = 'checkpoints'
t_cnfg.tb_dir = 'tensorboard'
t_cnfg.log_dir = 'logs'
t_cnfg.le = train.le
t_cnfg.max_step = 100000000
t_cnfg.batch_size = 150
t_cnfg.dropout_keep_prob = 0.5
t_cnfg.log_every = 50
t_cnfg.print_every = math.ceil(XY_train_valid[0].shape[0]/t_cnfg.batch_size)  # every epoch
t_cnfg.n_ave_ll_valid = 8
t_cnfg.start_step_early_stopping = 100000000
t_cnfg.early_stopping_patience =0.1

In [ ]:
print(g_cnfg)
print(t_cnfg)

GraphClass = globals()[graph_name]
graph = GraphClass(g_cnfg)
graph.train_model(t_cnfg, XY_train_valid)

name: graph_01
X_conv_h: 33
X_conv_w: 18
X_img_h: 129
X_img_w: 71
Y_vector_len: 31
conv1_kernel_size: 5
conv1_mp_size: 5
conv1_mp_strides: 2
conv1_n_filters: 16
conv2_kernel_size: 3
conv2_mp_size: 3
conv2_mp_strides: 2
conv2_n_filters: 32
lr_decay_rate: 0.7
lr_decay_steps: 1,000,000
lr_initial: 0.1
n_flat: 19,008
n_hidden1: 1,800
n_hidden2: 50

name: run_01
batch_size: 150
ckp_dir: checkpoints
dropout_keep_prob: 0.5
early_stopping_patience: 0.1
le: LabelEncoder()
log_dir: logs
log_every: 50
max_step: 100,000,000
n_ave_ll_valid: 8
print_every: 367
start_step_early_stopping: 100,000,000
tb_dir: tensorboard

Spectrogram2Conv_graph_01_run_01
Training starts @ 12/10/2017 00:19:43
Epoch 1 Step 1 Best model saved @ 12/10/2017 00:19:48 [Ave valid] 3.440
Epoch 1 Step 1 ends @ 12/10/2017 00:19:48 [Train] 3.422, 5.3% [Valid] 3.4% [Ave valid] 3.440
Epoch 1 Step 50 Best model saved @ 12/10/2017 00:21:23 [Ave valid] 3.401
Epoch 1 Step 100 Best model saved @ 12/10/2017 00:23:00 [Ave valid] 3.343
Epoc